TODO: downweight zero sales and use frequency on test store-item pair to train

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from pandas.tseries.offsets import MonthEnd
import cPickle

In [2]:
def isConsecutive(seq):
    # if non-consecutive, need to re-map to consecutive number starting from 1
    uniq = pd.unique(seq)
    return len(uniq) == (uniq.max()-uniq.min() + 1)

In [3]:
def isUniqBigger(seq1,seq2):
    return set(seq1) >= set(seq2)

In [4]:
def reMapDF(df,cols):
    # remap cols in dataframe to consecutive integers starting from one
    for col in cols:
        uniq = pd.unique(df[col])
        dict_ = {item:i+1 for i,item in enumerate(uniq)}
        df = df.replace({col:dict_})
    return df

In [5]:
def dimentionDF(df,cols):
    return {col:len(set(df[col])) for col in cols}

In [6]:
def mergeFillCast(df1,df2,key):
    cols = df2.columns.values
    types = df2.dtypes.values
    dict_ = {col:type_ for col,type_ in zip(cols,types)}
    dfOut = pd.merge(df1, df2, how='left', on=key, 
             suffixes=('', '_y'), copy=True, indicator=False).fillna(0)
    dfOut[cols] = \
        dfOut[cols].astype(dict_)
    return dfOut

In [7]:
def mergeFillCastsss(df0,dfs,keys):
    for df,key in zip(dfs,keys):
        df0 = mergeFillCast(df0,df,key)
    return df0

In [8]:
types = {'id': 'int32',
         'item_nbr': 'int32',
         'store_nbr': 'int8',
         'unit_sales': 'float32',
         'onpromotion': bool}

In [9]:
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)
train.loc[train.unit_sales<0,'unit_sales'] = .0 # clip negative sales to zero

In [11]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,2
1,2013-01-01,25,105574,1.0,2
2,2013-01-01,25,105575,2.0,2
3,2013-01-01,25,108079,1.0,2
4,2013-01-01,25,108701,1.0,2


In [50]:
test = pd.read_csv('test.csv',parse_dates=['date'],dtype=types, infer_datetime_format=True)
test = test.fillna(2,axis=1)
test.onpromotion = test.onpromotion.astype(np.int8)

Needs to map items2 before mapping item_nbr

In [12]:
items = pd.read_csv('items.csv')
stores = pd.read_csv('stores.csv')

In [13]:
items2 = reMapDF(items,['family','class'])
items2[['family','class','perishable']] = \
        items2[['family','class','perishable']].astype('int16')

In [14]:
stores2 = reMapDF(stores,['city', 'state', 'type'])
stores2 = stores2.astype('int8')

In [15]:
val = train[train.date >= '2017-07-31']

In [16]:
train = train[train.date < '2017-07-31']

In [17]:
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}
iter_mapping = lambda x: item_dict[x] if x in item_dict else 0

Create date related variables. Multiple holiday can be on the same date. Sample one holiday from many to create multiple holiday lookup.

In [18]:
holidays_events = pd.read_csv('holidays_events.csv',parse_dates=['date'],infer_datetime_format=True)

In [19]:
holidays_events_list = [holidays_events.groupby('date').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)\
                                .groupby('date').first().reset_index() for _ in range(10)]

In [20]:
holidays_events_list2 = [reMapDF(holiday.drop('description',1),['type', 'locale', 'locale_name'])\
                        for holiday in holidays_events_list]

In [21]:
for holiday in holidays_events_list2:
    holiday[['type', 'locale', 'locale_name','transferred']] = \
        holiday[['type', 'locale', 'locale_name','transferred']].astype('int8')

In [22]:
dateVar = pd.DataFrame(pd.date_range('2013-01-01', '2017-08-31'),columns=['date'])

In [23]:
dateVar['dayOfWeek'] = dateVar.date.dt.dayofweek

In [24]:
dateVar['payDay'] = ((dateVar.date.dt.day == dateVar.date.dt.days_in_month) | \
                     (dateVar.date.dt.day == 15)) * 1

In [25]:
dateVar['month'] = dateVar.date.dt.month

In [26]:
dateVar['earthquake'] = (dateVar.date > '2016-04-16') & (dateVar.date <= '2016-04-24')

In [27]:
dateVar.columns.values

array(['date', 'dayOfWeek', 'payDay', 'month', 'earthquake'], dtype=object)

In [28]:
dateVar[['dayOfWeek', 'payDay', 'month', 'earthquake']] = \
    dateVar[['dayOfWeek', 'payDay', 'month', 'earthquake']].astype('int8')

In [29]:
oil = pd.read_csv('oil.csv',parse_dates=['date'],infer_datetime_format=True)

In [30]:
dateVar = pd.merge(dateVar,oil,'left','date').fillna(method='bfill')

In [31]:
dateVar['dcoilwtico'] = (dateVar['dcoilwtico'] - np.mean(dateVar['dcoilwtico']))/np.std(dateVar['dcoilwtico'])

In [32]:
dateVar_list = [pd.merge(dateVar,holiday,'left','date').fillna(0)\
               for holiday in holidays_events_list2]

In [33]:
dateVar_list[2].head()

,date,dayOfWeek,payDay,month,earthquake,dcoilwtico,type,locale,locale_name,transferred
0,2013-01-01,1,0,1,0,0.993413,1.0,3.0,12.0,0.0
1,2013-01-02,2,0,1,0,0.993413,0.0,0.0,0.0,0.0
2,2013-01-03,3,0,1,0,0.986788,0.0,0.0,0.0,0.0
3,2013-01-04,4,0,1,0,0.992634,0.0,0.0,0.0,0.0
4,2013-01-05,5,0,1,0,0.995752,5.0,3.0,12.0,0.0


In [28]:
with open(r"dateVar.pickle", "wb") as output_file:
    cPickle.dump(dateVar_list, output_file)

In [45]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

Use the first date that store has sales on any items as the first date to densify the time series data.

In [18]:
def CreateData(data):
    SI_train_sales = data.groupby(['store_nbr','item_nbr'])[['date','unit_sales','onpromotion']].\
                    agg(lambda x: tuple(x)).reset_index()
    storeTime = data.groupby(['store_nbr'])['date'].agg([np.min,np.max]).reset_index()
    dfs = [items2,stores2,storeTime]
    keys = ['item_nbr','store_nbr','store_nbr']
    SI_train = mergeFillCastsss(SI_train_sales,dfs,keys)
    SI_train['item_nbr'] = SI_train.item_nbr.map(iter_mapping)
    SI_train['amin'] = pd.to_datetime(SI_train['amin'])
    SI_train['amax'] = pd.to_datetime(SI_train['amax'])
    return SI_train[['store_nbr',
                     'item_nbr',
                     'family',
                     'class',
                     'perishable',
                     'city',
                     'state',
                     'type',
                     'cluster',
                     'date',
                     'unit_sales', 
                     'onpromotion',
                     'amin',
                     'amax']]

In [19]:
trainRNN = CreateData(train)
valRNN = CreateData(val)

In [ ]:
print type(valRNN.date.loc[0])
print type(trainRNN.unit_sales.loc[0])
#print type(valRNN_Test.unit_sales.loc[0])

<type 'tuple'>
<type 'tuple'>


In [25]:
trainRNN.to_csv('trainRNN.csv',index=False)

In [33]:
valRNN.to_csv('valRNN.csv',index=False)

In [10]:
trainRNN = pd.read_csv('trainRNN.csv',engine='python')

In [75]:
valRNN_Test = pd.read_csv('valRNN.csv',engine='python')

In [81]:
def RNN_generator(data,batchSize,seqSize,df,key,discreteList,shuffle=True,downSample=1):
    # return bool (if init should be reset) 
    # and a list [y (B,T),weight (B,T),Xcontinue of shape (B,T,2)] + [Xdiscrete] of shape (B,T) + [X] of shape (B,)
    # finetune by use store-item pair in testset only
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    n = data.shape[0]

    for from_ in range(0,n-batchSize,batchSize):
        X = list(data.loc[from_:from_+batchSize-1,'store_nbr':'cluster'].values.astype(np.int32).T)
        weight = np.ones((batchSize,1),dtype=np.float32)
        weight[data.loc[from_:from_+batchSize-1,'perishable']==1] = 1.25
        for j,(y,w,Xdiscrete,Xcontinue) in enumerate(timeGenerator(\
                           data.loc[from_:from_+batchSize-1,'date':'amax'],seqSize,downSample,df,key,discreteList)):
            yield j==0, [y,weight*w,Xcontinue] + Xdiscrete + X
            
def timeGenerator(data,seqSize,downSample,df,key,discreteList):
    # df is time related variables like holiday and seasonality
    # returns y, weight, X_discrete, X_continue
    n = data.shape[0]
    data['curr'] = data.amin 
    sparse = pd.concat([data.apply(lambda x: pd.Series(x.date,name='date'),axis=1)\
                                    .stack().reset_index().drop(['level_1'],1),\
                        data.apply(lambda x: pd.Series(x.unit_sales),axis=1)\
                                    .stack().reset_index().drop(['level_0','level_1'],1),\
                        data.apply(lambda x: pd.Series(x.onpromotion),axis=1)\
                                    .stack().reset_index().drop(['level_1'],1)],1)
    sparse.columns = ['level','date','sales','onpromotion']
    while np.all(data.curr + pd.DateOffset(seqSize) <= data['amax']):
        dense = data.apply(lambda x:pd.Series(pd.date_range(x.curr,periods=seqSize+1)),axis=1)\
                            .stack().reset_index().drop(['level_1'],1)
        dense.columns = ['level','date']
        dense = pd.merge(pd.merge(dense, df[np.random.randint(10)], how='left', on=key),
                        sparse,how='left',on=['level','date']).fillna(0)
        dense_continue = dense[['dcoilwtico','sales']].values.astype(np.float32)\
                                  .reshape((n,seqSize+1,2))[:,:seqSize,:]
        dense_discrete = list(np.moveaxis(dense[discreteList].values.astype(np.int32)\
                                                .reshape((n,seqSize+1,len(discreteList)))[:,:seqSize,:]\
                                          ,2,0))
        y = dense['sales'].values.astype(np.float32).reshape((n,seqSize+1))[:,1:]
        weight = np.ones_like(y,dtype=np.float32)
        weight[y==0] = downSample
        yield y, weight, dense_discrete, dense_continue
        data.curr = data.curr + pd.DateOffset(seqSize)   

In [82]:
# of shape (B,T)
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [83]:
# of shape (B,)
Xlist = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [84]:
g = RNN_generator(valRNN,5,3,dateVar_list,'date',discreteList,shuffle=True,downSample=0.8)

In [85]:
a,b = g.next()

AttributeError: ("'Series' object has no attribute 'onpromotion'", u'occurred at index 0')

In [57]:
a

True

In [58]:
b[0]

array([[  0.,   1.,   0.],
       [  0.,   0.,   0.],
       [  3.,   6.,  10.],
       [  5.,   3.,   5.],
       [  3.,   3.,   3.]], dtype=float32)

In [59]:
b[1]

array([[ 0.80000001,  1.        ,  0.80000001],
       [ 0.80000001,  0.80000001,  0.80000001],
       [ 1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ],
       [ 1.25      ,  1.25      ,  1.25      ]], dtype=float32)

In [60]:
b[2][:,:,0]

array([[-0.67974991, -0.71950358, -0.70352417],
       [-0.67974991, -0.71950358, -0.70352417],
       [-0.67974991, -0.71950358, -0.70352417],
       [-0.67974991, -0.71950358, -0.70352417],
       [-0.67974991, -0.71950358, -0.70352417]], dtype=float32)

In [61]:
b[2][:,:,1]

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 4.,  3.,  6.],
       [ 2.,  5.,  3.],
       [ 5.,  3.,  3.]], dtype=float32)

In [62]:
b[3]

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]], dtype=int32)

In [63]:
b[10]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)

In [64]:
b[11:]

[array([ 7, 51, 49, 41, 34], dtype=int32),
 array([1638, 1246, 2581,  544, 3753], dtype=int32),
 array([ 1,  9, 29,  1, 11], dtype=int32),
 array([  7,  48, 280,  38, 199], dtype=int32),
 array([0, 0, 0, 0, 1], dtype=int32),
 array([ 1, 10,  1, 19, 10], dtype=int32),
 array([ 1,  9,  1, 14,  9], dtype=int32),
 array([1, 5, 5, 1, 2], dtype=int32),
 array([ 8, 17, 11,  4,  6], dtype=int32)]